In [1]:
from selenium import webdriver
import pandas as pd
from scrap_article import scrap_article
from threading import Thread

from datetime import datetime as dt
from datetime import timedelta as td

In [2]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')

In [3]:
chrome = webdriver.Chrome(options=options)

In [4]:
chrome.get(url = 'http://wyborcza.pl/0,75248.html')

In [5]:
entries = []
for entry in chrome\
    .find_element_by_xpath('//*[@id="pagetype_index"]/section[5]/div/main/div/div[2]/div/div[2]/ul')\
    .find_elements_by_class_name('entry'):
    entryContainer = {}
    
    entryContainer['author'] = entry.find_element_by_class_name('author').text.replace('|', '').strip()
    header = entry.find_element_by_tag_name('h3')\
                .find_element_by_tag_name('a')
    entryContainer['title'] = header.get_attribute('title')
    entryContainer['url'] = header.get_attribute('href')
    entryContainer['article_id'] = entryContainer['url'].split('/')[-1].split(',')[2]
    entryContainer['lead'] = entry.find_element_by_tag_name('p').text
    entryContainer['date'] = entry.find_element_by_class_name('when').text
    
    entries.append(entryContainer)

In [6]:
scrapped_articles = pd.DataFrame(entries)
scrapped_articles['is_new'] = True
scrapped_articles['content'] = None

In [7]:
scrapped_articles.head(5)

,article_id,author,date,lead,title,url,is_new,content
0,24797627,Ewa Ivanova,17-05-2019 21:22,Żaden sąd nie jest właściwy do kontrolowania w...,"Wyborcza pisze o pozwie, prezydent Duda odpowiada","http://wyborcza.pl/7,75398,24797627,wyborcza-p...",True,None
1,24797255,Paweł Rutkiewicz,17-05-2019 19:40,- My w tych wyborach możemy zrobić pierwszy kr...,Wybory europejskie 2019. Kaczyński w Łodzi. Pi...,"http://lodz.wyborcza.pl/lodz/7,35136,24797255,...",True,None
2,24796693,mc,17-05-2019 17:34,Ksiądz z diecezji bydgoskiej uwikłany w pedofi...,Kapłan uwikłany w pedofilię przebywa w bydgosk...,"http://bydgoszcz.wyborcza.pl/bydgoszcz/7,48722...",True,None
3,24797373,Marta Kielczewska-Konopka,17-05-2019 19:32,"Diagnoza stanu paryskiej katedry, zdewastowane...",Odbudowa Notre Dame: 850 mln euro wystarczy? A...,"http://wyborcza.pl/7,75399,24797373,odbudowa-n...",True,None
4,24797222,Bartosz T. Wieliński,17-05-2019 19:11,"Koniec z pokorą, cierpliwością, służbą? Niemie...",Maria 2.0 wstrząsa niemieckim Kościołem,"http://wyborcza.pl/7,75399,24797222,maria-2-0-...",True,None


In [8]:
existing_articles = pd.read_csv('articles.csv')

In [9]:
existing_articles.head(5)

,author,date,lead,title,url
0,Piotr Andrusieczko,16-05-2019 16:51,W czwartek Rada Najwyższa Ukrainy wyznaczyła i...,Data inauguracji Wołodymyra Zełenskiego w końc...,"http://wyborcza.pl/7,75399,24793013,data-inaug..."
1,Magdalena Mach,16-05-2019 16:26,Policjanci zatrzymali mężczyznę podejrzanego o...,Atakował kobiety w parku w centrum Rzeszowa. B...,"http://rzeszow.wyborcza.pl/rzeszow/7,34962,247..."
2,Jarosław Bińczyk,16-05-2019 16:35,Z siatkarskiej PlusLigi znikają znani włoscy t...,Schyłek ery w siatkówce? Polska jako etap prze...,"http://wyborcza.pl/7,154903,24791353,schylek-e..."
3,Maciej Stasiński,16-05-2019 15:47,To kolejna próba namówienia obu stron - od trz...,W Norwegii poufne rokowania z reżimem Nicolása...,"http://wyborcza.pl/7,75399,24792543,w-norwegii..."
4,key,16-05-2019 15:34,Coraz więcej trudnych pytań do bydgoskiego bis...,Bydgoski poseł pyta biskupa Tyrawę o jeszcze j...,"http://bydgoszcz.wyborcza.pl/bydgoszcz/7,48722..."


In [10]:
combined_articles = existing_articles.append(scrapped_articles, sort=False)

In [11]:
len(combined_articles)

30

In [12]:
len(combined_articles.drop_duplicates(keep='first'))

30

In [13]:
all_articles = combined_articles.drop_duplicates(keep='first')

In [14]:
all_articles['date'] = pd.to_datetime(all_articles['date'])

In [17]:
articles_to_drop = all_articles[
    all_articles['date'] <= dt.now() - td(days=7)
]

In [18]:
if articles_to_drop.empty == True:
    pass
else:
    existing_articles_to_drop = pd.read_csv('data/old_articles.csv')
    existing_articles_to_drop = existing_articles_to_drop.append(articles_to_drop)
    existing_articles_to_drop.to_csv('data/old_articles.csv', index=False)

In [19]:
articles_to_continue = all_articles[
    all_articles['date'] > dt.now() - td(days=7)
]

In [24]:
# TMP
articles_to_continue['is_new'] = True
articles_to_continue['last_date'] = dt.now() - td(minutes=60)

In [25]:
articles_to_continue.head(1)

,author,date,lead,title,url,article_id,is_new,content,last_date
0,Piotr Andrusieczko,2019-05-16 16:51:00,W czwartek Rada Najwyższa Ukrainy wyznaczyła i...,Data inauguracji Wołodymyra Zełenskiego w końc...,"http://wyborcza.pl/7,75399,24793013,data-inaug...",NaN,True,NaN,2019-05-17 21:02:33.449935


In [26]:
def check_date(last_date, pub_date):
    hours_since_pub = (dt.now() - pub_date).total_seconds() / 60 / 60
    if hours_since_pub < 2:
        interval = 10 # in minutes
    elif hours_since_pub < 5:
        interval = 15
    elif hours_since_pub < 10:
        interval = 20
    else:
        interval = 30
    
    if dt.now() - td(minutes=interval) < last_date:
        return True
    else:
        return False

In [27]:
articles_list = articles_to_continue.to_dict('rows')

In [48]:
container = []
for article in articles_list:
    article_id = article['url'].split('/')[-1].split(',')[2]
    if article['is_new'] == True:
        a = scrap_article(chrome=chrome, url=article['url'], first_time=True)
        article['article_id'] = article_id
        article['content'] = a['content']
        article['division'] = a['division']
        article['highlight'] = a['highlight']
        article['media_type'] = a['media_type']
        article['media_src'] = a['media_src']
        article['media_desc'] = a['media_desc']
        
        article['is_new'] = False
        
    else:
        if check_date(article['last_date'], article['date']):
            a = scrap_article(chrome=chrome, url=article['url'], first_time=False)
        else:
            continue
    
    container.append(article)
    with open('data/comments/'+article_id, 'w') as f:
        f.write(str({'row': {
            'comments': a['comments'],
            'timestamp': dt.timestamp(dt.now())
            }
        }))
    break

In [35]:
articles_updated = pd.DataFrame(container)

In [36]:
articles_updated.head()

,article_id,author,content,date,division,highlight,is_new,last_date,lead,media_desc,media_src,media_type,title,url
0,24792543,Maciej Stasiński,Na rozmowy mediacyjne do Oslo zaprosił Wenezue...,2019-05-16 15:47:00,Świat,To kolejna próba namówienia obu stron - od trz...,False,2019-05-17 21:02:33.449935,To kolejna próba namówienia obu stron - od trz...,"Przywódca wenezuelskiej opozycji, tymczasowy p...","https://bi.im-g.pl/im/41/a4/17/z24792129V,Przy...",image,W Norwegii poufne rokowania z reżimem Nicolása...,"http://wyborcza.pl/7,75399,24792543,w-norwegii..."
